# Importing & Loading Dependencies

In [1]:
!pip install monai
# !pip install 'monai[nibabel]'

import nibabel as nib
from monai.transforms import LoadImage, Compose, NormalizeIntensityd, RandSpatialCropd, RandFlipd, \
                             RandRotate90d, Rand3DElasticd, RandAdjustContrastd, CenterSpatialCropd,\
                             Resized, RandRotated, RandZoomd, RandGaussianNoised, Spacingd, RandShiftIntensityd,  CropForegroundd, SpatialPadd, AsDiscrete, GridPatchd\

import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch
import logging
import numpy as np
import os
import random
from typing import Optional, Sequence, Tuple, Union
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import interpolate
import pdb

from monai.networks.blocks.convolutions import Convolution
from monai.networks.layers.factories import Act, Norm
from monai.networks.layers.utils import get_act_layer, get_norm_layer
from monai.networks import one_hot
from monai.metrics import DiceMetric, HausdorffDistanceMetric
import torchvision
import math

from grpc import insecure_channel
import argparse
from torch import optim, amp
from monai.losses import DiceLoss,BarlowTwinsLoss
import torch.distributed as dist

from pathlib import Path
import argparse
import json
import math
import os
import random
import signal
import subprocess
import sys
import time
import pdb
import logging
from ipywidgets import interact, IntSlider

from monai.losses import DiceLoss
from torch import nn, optim
from tqdm import tqdm
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.2 MB/s eta 0:00:00


# Creating Dataset with Preprocessing

In [2]:
class CustomDataset3D(Dataset):
    def __init__(
        self,
        data_dirs,
        patient_lists,
        mode
        ):

        self.data_dirs = data_dirs
        self.patient_lists = patient_lists
        self.mode = mode

    @staticmethod
    def resize_with_aspect_ratio(keys, target_size):
        def transform(data):
            for key in keys:
                volume = data[key]
                original_shape = volume.shape[-3:]
    
                scaling_factor = min(
                    target_size[0] / original_shape[0],
                    target_size[1] / original_shape[1],
                    target_size[2] / original_shape[2]
                )
    
                # Computing the intermediate size while preserving aspect ratio
                new_shape = [
                    int(dim * scaling_factor) for dim in original_shape
                ]
    
                # Resizing to the intermediate shape
                resize_transform = Resized(keys=[key], spatial_size=new_shape, mode="trilinear" if key == "imgs" else "nearest-exact")
                data = resize_transform(data)
    
                # Padding to the final target size
                pad_transform = SpatialPadd(keys=[key], spatial_size=target_size, mode="constant")
                data = pad_transform(data)
            return data

        return transform

    def preprocess(cls, data, mode):
        if mode == 'training':
          transform = Compose([
            CropForegroundd(keys=["imgs", "masks"], source_key="imgs"),
            cls.resize_with_aspect_ratio(keys=["imgs", "masks"], target_size=[128, 128, 128]),
            NormalizeIntensityd( keys=['imgs'], nonzero=False, channel_wise=True),
              
            RandFlipd(keys=["imgs", "masks"],   
                    prob=0.5,                 
                    spatial_axis=2,  
            ),

            RandAdjustContrastd(
                keys=["imgs"],          
                prob=0.15,             
                gamma=(0.65, 1.5),   
            ),
            
        ])

        elif mode == 'validation':
          transform = Compose([
            CropForegroundd(keys=["imgs", "masks"], source_key="imgs"),
            cls.resize_with_aspect_ratio(keys=["imgs", "masks"], target_size=[128, 128, 128]),
            NormalizeIntensityd( keys=['imgs'], nonzero=False, channel_wise=True)

        ])

        else: # 'testing'
          transform = Compose([
            CropForegroundd(keys=["imgs", "masks"], source_key="imgs"),
            cls.resize_with_aspect_ratio(keys=["imgs", "masks"], target_size=[128, 128, 128]),
            NormalizeIntensityd( keys=['imgs'], nonzero=False, channel_wise=True)

        ])

        augmented_data = transform(data)
        return augmented_data
        
    def __len__(self):
        return len(self.patient_lists)

    def __getitem__(self, idx):
        patient_id = self.patient_lists[idx]
        loadimage = LoadImage(reader='NibabelReader', image_only=True)

        data_type=patient_id.split('-')[1]
        if data_type == 'GLI':
            patient_folder_path = os.path.join('/kaggle/input/bratsglioma/Training', patient_id)
        elif data_type == 'SSA':
            patient_folder_path = os.path.join('/kaggle/input/bratsafrica24', patient_id)
        elif data_type == 'PED':
            patient_folder_path = os.path.join('/kaggle/input/bratsped/Training', patient_id)
        elif data_type == 'MEN':
            patient_folder_path = os.path.join('/kaggle/input/bratsmen', patient_id)
        else:
            patient_folder_path = os.path.join('/kaggle/input/bratsmet24', patient_id)

        def resolve_file_path(folder, name):
            file_path = os.path.join(folder, name)
            # Check if the given path is a directory (case with 4 subdirs)
            if os.path.isdir(file_path):
                # Find the first file inside the directory that ends with .nii
                for root, _, files in os.walk(file_path):
                    for file in files:
                        if file.endswith(".nii"):
                            return os.path.join(root, file)
            return file_path


        # Resolve paths for all required image types
        t1c_path  = resolve_file_path(patient_folder_path, patient_id + '-t1c.nii')
        t1n_path  = resolve_file_path(patient_folder_path, patient_id + '-t1n.nii')
        t2f_path  = resolve_file_path(patient_folder_path, patient_id + '-t2f.nii')
        t2w_path  = resolve_file_path(patient_folder_path, patient_id + '-t2w.nii')
        seg_path  = os.path.join(patient_folder_path, patient_id + '-seg.nii')

        t1c_loader   = loadimage( t1c_path )
        t1n_loader   = loadimage( t1n_path )
        t2f_loader   = loadimage( t2f_path )
        t2w_loader   = loadimage( t2w_path )
        masks_loader = loadimage( seg_path )

        # Make the dimension of channel
        t1c_tensor   = torch.Tensor(t1c_loader).unsqueeze(0)
        t1n_tensor   = torch.Tensor(t1n_loader).unsqueeze(0)
        t2f_tensor   = torch.Tensor(t2f_loader).unsqueeze(0)
        t2w_tensor   = torch.Tensor(t2w_loader).unsqueeze(0)
        masks_tensor = torch.Tensor(masks_loader).unsqueeze(0)

        concat_tensor = torch.cat( (t1c_tensor, t1n_tensor, t2f_tensor, t2w_tensor, masks_tensor), 0 )
        data = {            
            'imgs'  : np.array(concat_tensor[0:4,:,:,:]),
            'masks' : np.array(concat_tensor[4:,:,:,:])
        }

        augmented_imgs_masks = self.preprocess(data, self.mode)
        imgs  = np.array(augmented_imgs_masks['imgs'])
        masks = np.array(augmented_imgs_masks['masks'])

        y = {

            'imgs'  : torch.from_numpy(imgs).type(torch.FloatTensor),
            'masks' : torch.from_numpy(masks).type(torch.FloatTensor),
            'patient_id' : patient_id,
            'data_type' : data_type

        }

        return y

# Data Loaders

In [3]:
def combine_datasets(dataset_lists, batch_size=2):
    min_len = min(len(dataset) for dataset in dataset_lists)
    combined_paths = []
    
    for i in range(0, min_len, batch_size):
        for dataset in dataset_lists:
            batch = dataset[i:i + batch_size]
            if len(batch)==batch_size:
                combined_paths.extend(batch)
            else: 
                break
                
    return combined_paths

In [4]:
def prepare_data_loaders(args):
    train_datasets, val_datasets, test_datasets = [], [], []
    split_ratio = {'training': 0.71, 'validation': 0.09, 'testing': 0.2}
    
    for i, data_dir in enumerate(args['data_dirs']):
        patient_lists = os.listdir( data_dir )
        patient_lists.sort()
        total_patients = len(patient_lists)

        random.seed(5)
        random.shuffle(patient_lists)
    
        train_split = int(split_ratio['training'] * total_patients)
        val_split = int(split_ratio['validation'] * total_patients)
    
        train_patient_lists = patient_lists[:train_split]
        val_patient_lists = patient_lists[train_split : train_split + val_split]
        test_patient_lists = patient_lists[train_split + val_split :]
    
        train_patient_lists.sort()
        val_patient_lists.sort()
        test_patient_lists.sort()

        # print(f'Train IDs of {data_dir}', train_patient_lists)
        # print(f'Val IDs of {data_dir}', val_patient_lists)
        print(f'Test IDs of {data_dir}', test_patient_lists)
        
        print(f'Number of training samples in {data_dir.split("/")[3]} DataSet: {len(train_patient_lists)}')
        print(f'Number of validation samples in {data_dir.split("/")[3]} DataSet: {len(val_patient_lists)}')
        print(f'Number of testing samples in {data_dir.split("/")[3]} DataSet: {len(test_patient_lists)} ')

        train_datasets.append(train_patient_lists)
        val_datasets.append(val_patient_lists)
        test_datasets.append(test_patient_lists)
            
    combined_trainDataset = combine_datasets(train_datasets, batch_size=args['train_batch_size'])
    combined_valDataset = combine_datasets(val_datasets, batch_size=args['val_batch_size'])
    combined_testDataset = combine_datasets(test_datasets, batch_size=args['test_batch_size'])
    
    print(f'Number of combined training samples', len(combined_trainDataset))
    print(f'Number of combined validation samples', len(combined_valDataset))
    print(f'Number of combined testing samples', len(combined_testDataset))
    
    trainDataset = CustomDataset3D( args['data_dirs'], combined_trainDataset, mode='training')
    valDataset = CustomDataset3D( args['data_dirs'], combined_valDataset, mode='validation')
    testDataset = CustomDataset3D( args['data_dirs'], combined_testDataset, mode='testing')
    
    trainLoader = DataLoader(
        trainDataset, batch_size=args['train_batch_size'], num_workers=args['workers'], prefetch_factor=2,
        pin_memory=True, shuffle=False)
    
    valLoader = DataLoader(
        valDataset, batch_size=args['val_batch_size'], num_workers=args['workers'], prefetch_factor=2,
        pin_memory=True, shuffle=False)
    
    testLoader = DataLoader(
        testDataset, batch_size=args['test_batch_size'], num_workers=args['workers'], prefetch_factor=2,
        pin_memory=True, shuffle=False)

    return trainLoader, valLoader, testLoader

# DynUNet Model

In [5]:
class UnetBasicBlock(nn.Module):
    """
    A CNN module module that can be used for DynUNet, based on:
    `Automated Design of Deep Learning Methods for Biomedical Image Segmentation <https://arxiv.org/abs/1904.08128>`_.
    `nnU-Net: Self-adapting Framework for U-Net-Based Medical Image Segmentation <https://arxiv.org/abs/1809.10486>`_.

    Args:
        spatial_dims: number of spatial dimensions.
        in_channels: number of input channels.
        out_channels: number of output channels.
        kernel_size: convolution kernel size.
        stride: convolution stride.
        norm_name: feature normalization type and arguments.
        act_name: activation layer type and arguments.
        dropout: dropout probability.

    """

    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[Sequence[int], int],
        stride: Union[Sequence[int], int],
        norm_name: Union[Tuple, str] = ("INSTANCE", {"affine": True}),
        act_name: Union[Tuple, str] = ("leakyrelu", {"inplace": True, "negative_slope": 0.01}),
        dropout: Optional[Union[Tuple, str, float]] = None,
    ):
        super().__init__()
        self.conv1 = get_conv_layer(
            spatial_dims,
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            dropout=dropout,
            conv_only=True,
        )

        self.conv2 = get_conv_layer(
            spatial_dims,
            out_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=1,
            dropout=dropout,
            conv_only=True
        )
        self.lrelu = get_act_layer(name=act_name)
        self.norm1 = get_norm_layer(name=norm_name, spatial_dims=spatial_dims, channels=out_channels)
        self.norm2 = get_norm_layer(name=norm_name, spatial_dims=spatial_dims, channels=out_channels)

    def forward(self, inp):
        out = self.conv1(inp)
        out = self.norm1(out)
        out = self.lrelu(out)
        out = self.conv2(out)
        out = self.norm2(out)
        out = self.lrelu(out)
        return out



class UnetUpBlock(nn.Module):
    """
    An upsampling module that can be used for DynUNet, based on:
    `Automated Design of Deep Learning Methods for Biomedical Image Segmentation <https://arxiv.org/abs/1904.08128>`_.
    `nnU-Net: Self-adapting Framework for U-Net-Based Medical Image Segmentation <https://arxiv.org/abs/1809.10486>`_.

    Args:
        spatial_dims: number of spatial dimensions.
        in_channels: number of input channels.
        out_channels: number of output channels.
        kernel_size: convolution kernel size.
        stride: convolution stride.
        upsample_kernel_size: convolution kernel size for transposed convolution layers.
        norm_name: feature normalization type and arguments.
        act_name: activation layer type and arguments.
        dropout: dropout probability.
        trans_bias: transposed convolution bias.

    """

    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        out_channels: int,
        kernel_size: Union[Sequence[int], int],
        upsample_kernel_size: Union[Sequence[int], int],
        norm_name: Union[Tuple, str] = ("INSTANCE", {"affine": True}),
        act_name: Union[Tuple, str] = ("leakyrelu", {"inplace": True, "negative_slope": 0.01}),
        dropout: Optional[Union[Tuple, str, float]] = None,
        trans_bias: bool = False,
    ):
        super().__init__()
        upsample_stride = upsample_kernel_size
        
        # ( a purple arrow in the paper )
        self.transp_conv = get_conv_layer(
            spatial_dims,
            in_channels,
            out_channels,
            kernel_size=upsample_kernel_size,
            stride=upsample_stride,
            dropout=dropout,
            bias=trans_bias,
            conv_only=True,
            is_transposed=True,
        )
        
        # A light blue conv blocks in the decoder of nnUNet
        self.conv_block = UnetBasicBlock(
            spatial_dims,
            out_channels + out_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=1,
            dropout=dropout,
            norm_name=norm_name,
            act_name=act_name,
        )

    def forward(self, inp, skip):
        # number of channels for skip should equals to out_channels
        out = self.transp_conv(inp)
        out = torch.cat((out, skip), dim=1)
        out = self.conv_block(out)
        return out



class UnetOutBlock(nn.Module):
    def __init__(
        self, spatial_dims: int, in_channels: int, out_channels: int, dropout: Optional[Union[Tuple, str, float]] = None
    ):
        super().__init__()
        self.conv = get_conv_layer(
            spatial_dims, in_channels, out_channels, kernel_size=1, stride=1, dropout=dropout, bias=True, conv_only=True
        )

    def forward(self, inp):
        return self.conv(inp)
    

def get_conv_layer(
    spatial_dims: int,
    in_channels: int,
    out_channels: int,
    kernel_size: Union[Sequence[int], int] = 3,
    stride: Union[Sequence[int], int] = 1,
    act: Optional[Union[Tuple, str]] = Act.PRELU,
    norm: Union[Tuple, str] = Norm.INSTANCE,
    dropout: Optional[Union[Tuple, str, float]] = None,
    bias: bool = False,
    conv_only: bool = True,
    is_transposed: bool = False,
):
    padding = get_padding(kernel_size, stride)
    output_padding = None
    if is_transposed:
        output_padding = get_output_padding(kernel_size, stride, padding)
    
    return Convolution(
        spatial_dims,
        in_channels,
        out_channels,
        strides=stride,
        kernel_size=kernel_size,
        act=act,
        norm=norm,
        dropout=dropout,
        bias=bias,
        conv_only=conv_only,
        is_transposed=is_transposed,
        padding=padding,
        output_padding=output_padding,
    )


def get_padding(
    kernel_size: Union[Sequence[int], int], stride: Union[Sequence[int], int]
) -> Union[Tuple[int, ...], int]:

    kernel_size_np = np.atleast_1d(kernel_size)
    stride_np = np.atleast_1d(stride)
    padding_np = (kernel_size_np - stride_np + 1) / 2
    if np.min(padding_np) < 0:
        raise AssertionError("padding value should not be negative, please change the kernel size and/or stride.")
    padding = tuple(int(p) for p in padding_np)

    return padding if len(padding) > 1 else padding[0]


def get_output_padding(
    kernel_size: Union[Sequence[int], int], stride: Union[Sequence[int], int], padding: Union[Sequence[int], int]
) -> Union[Tuple[int, ...], int]:
    kernel_size_np = np.atleast_1d(kernel_size)
    stride_np = np.atleast_1d(stride)
    padding_np = np.atleast_1d(padding)

    out_padding_np = 2 * padding_np + stride_np - kernel_size_np
    if np.min(out_padding_np) < 0:
        raise AssertionError("out_padding value should not be negative, please change the kernel size and/or stride.")
    out_padding = tuple(int(p) for p in out_padding_np)

    return out_padding if len(out_padding) > 1 else out_padding[0]

def set_requires_grad(nets, requires_grad=False):
    if not isinstance(nets, list):
        nets = [nets]
    for net in nets:
        if net is not None:
            for param in net.parameters():
                param.requires_grad = requires_grad

In [6]:
class DynUNet(nn.Module):
    def __init__(
        self,
        spatial_dims: int,
        in_channels: int,
        out_channels: int,
        deep_supervision: bool,
        KD: bool = False
    ):
        super().__init__()
        self.spatial_dims = spatial_dims
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.deep_supervision = deep_supervision
        self.KD_enabled = KD
        
        self.input_conv = UnetBasicBlock( spatial_dims=self.spatial_dims,
                                     in_channels=self.in_channels,
                                     out_channels=64,
                                     kernel_size=3,
                                     stride=1
                                     )
        self.down1 = UnetBasicBlock( spatial_dims=self.spatial_dims,
                                     in_channels=64,
                                     out_channels=96,
                                     kernel_size=3,
                                     stride=2 # Reduces spatial dims by 2
                                     )
        self.down2 = UnetBasicBlock( spatial_dims=self.spatial_dims,
                                     in_channels=96,
                                     out_channels=128,
                                     kernel_size=3,
                                     stride=2
                                     )
        self.down3 = UnetBasicBlock( spatial_dims=self.spatial_dims,
                                     in_channels=128,
                                     out_channels=192,
                                     kernel_size=3,
                                     stride=2
                                     )
        self.down4 = UnetBasicBlock( spatial_dims=self.spatial_dims,
                                     in_channels=192,
                                     out_channels=256,
                                     kernel_size=3,
                                     stride=2
                                     )
        self.down5 = UnetBasicBlock( spatial_dims=self.spatial_dims,
                                     in_channels=256,
                                     out_channels=384,
                                     kernel_size=3,
                                     stride=2
                                     )
        self.bottleneck = UnetBasicBlock( spatial_dims=self.spatial_dims,
                                     in_channels=384,
                                     out_channels=512,
                                     kernel_size=3,
                                     stride=2
                                     )
        self.up1 = UnetUpBlock( spatial_dims=self.spatial_dims,
                                in_channels=512,
                                out_channels=384,
                                kernel_size=3,
                                upsample_kernel_size=2
                                )
        self.up2 = UnetUpBlock( spatial_dims=self.spatial_dims,
                                in_channels=384,
                                out_channels=256,
                                kernel_size=3,
                                upsample_kernel_size=2
                                )
        self.up3 = UnetUpBlock( spatial_dims=self.spatial_dims,
                                in_channels=256,
                                out_channels=192,
                                kernel_size=3,
                                upsample_kernel_size=2
                                )
        self.up4 = UnetUpBlock( spatial_dims=self.spatial_dims,
                                in_channels=192,
                                out_channels=128,
                                kernel_size=3,
                                upsample_kernel_size=2
                                )
        
        self.up5 = UnetUpBlock( spatial_dims=self.spatial_dims,
                                in_channels=128,
                                out_channels=96,
                                kernel_size=3,
                                upsample_kernel_size=2
                                )        
        self.up6 = UnetUpBlock( spatial_dims=self.spatial_dims,
                                in_channels=96,
                                out_channels=64,
                                kernel_size=3,
                                upsample_kernel_size=2
                                )
        self.out1 = UnetOutBlock( spatial_dims=self.spatial_dims,
                                  in_channels=64,
                                  out_channels=self.out_channels,
                                  )
        self.out2 = UnetOutBlock( spatial_dims=self.spatial_dims,
                                  in_channels=96,
                                  out_channels=self.out_channels,
                                  )
        self.out3 = UnetOutBlock( spatial_dims=self.spatial_dims,
                                  in_channels=128,
                                  out_channels=self.out_channels,
                                  )
        
    def forward( self, input ):
        
        # Input
        x0 = self.input_conv( input ) # x0.shape = (B x 64 x 128 x 128 x 128)
        
        # Encoder
        x1 = self.down1( x0 ) # x1.shape = (B x 96 x 64 x 64 x 64) 
        x2 = self.down2( x1 ) # x2.shape = (B x 128 x 32 x 32 x 32)
        x3 = self.down3( x2 ) # x3.shape = (B x 192 x 16 x 16 x 16)
        x4 = self.down4( x3 ) # x4.shape = (B x 256 x 8 x 8 x 8)   
        x5 = self.down5( x4 ) # x5.shape = (B x 384 x 4 x 4 x 4)   
        
        # Bottleneck
        x6 = self.bottleneck( x5 ) # x6.shape = (B x 512 x 2 x 2 x 2)
        
        # Decoder
        x7  = self.up1( x6, x5 )  # x7.shape  = (B x 384 x 4 x 4 x 4)
        x8  = self.up2( x7, x4 )  # x8.shape  = (B x 256 x 8 x 8 x 8)
        x9  = self.up3( x8, x3 )  # x9.shape  = (B x 192 x 16 x 16 x 16)
        x10 = self.up4( x9, x2 )  # x10.shape = (B x 128 x 32 x 32 x 32)
        x11 = self.up5( x10, x1 ) # x11.shape = (B x 96 x 64 x 64 x 64)
        x12 = self.up6( x11, x0 ) # x12.shape = (B x 64 x 128 x 128 x 128)
        
        # Output
        output1 = self.out1( x12 )
        
        if (self.training and self.deep_supervision) or self.KD_enabled:
            
            # output['pred'].shape = B x 3 x 4 x 128 x 128 x 128
            output2 = interpolate( self.out2( x11 ), output1.shape[2:])
            output3 = interpolate( self.out3( x10 ), output1.shape[2:])
            output_all = [ output1, output2, output3 ]
            return { 'pred' : torch.stack(output_all, dim=1),
                     'bottleneck_feature_map' : x6 }
        
        return { 'pred' : output1 }

# Preparing ensemble

In [7]:
# from scipy.stats import mode

def generate_prediction_mask(pred):
    output_probs = (torch.sigmoid(pred) > 0.5)
    output = output_probs[0, 1:] # Get the only element in the batch (first one)
    return output.float()

def load_model(model_path):
    model_path = Path(model_path)
    model = DynUNet( spatial_dims=3, in_channels=4, out_channels=4, deep_supervision=False).to('cuda')       
    if (model_path).is_file():
        ckpt = torch.load(model_path, map_location='cuda', weights_only=True) #map_location='cuda' de momken t3mlak moshkla bs sebha law zabta
        model.load_state_dict(ckpt['teacher_model'])
        print(f"Loaded model: {model_path}")
    
    return model

def run_ensemble_fast(input_imgs, teacher_models):
    teacher_outputs = []
    
    for i in range(len(teacher_models)):
        model = load_model(teacher_models[i])
        model.eval()
        output = model(input_imgs)
        temp = generate_prediction_mask(output['pred'])
        teacher_outputs.append(temp)
    
    print("Shape of each teacher output:", teacher_outputs[0].shape)
    
    final_output = []
    for c_idx, channel_preds in enumerate(zip(*teacher_outputs)):
        stacked = torch.stack(channel_preds, dim=0)     # (N, H, W, D) N=5
        print(f"Voting by majority for channel {c_idx}..")
        m, _   = torch.mode(stacked, dim=0)             # (H, W, D)
        final_output.append(m)
    
    ensemble = torch.stack(final_output, axis=0)            # (C, H, W, D) C=3
    ensemble = ensemble.unsqueeze(0)
    print("Final ensemble predicion:", ensemble.shape)  # → (1, 3, 128, 128, 128)

    return ensemble

def prepare_teachers():
    teacher_model_paths = {
            'GLI': '/kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth',
            'SSA': '/kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080_valLoss_0.5561.pth',
            'PED': '/kaggle/input/pednewlabel/Teacher_model_after_epoch_99_trainLoss_1.4512_valLoss_1.0042.pth',
            'MEN': '/kaggle/input/meningiomateachernewlabels/Teacher_model_after_epoch_85_trainLoss_0.5824_valLoss_0.3318.pth',
            'MET': '/kaggle/input/met-teacher-new-labels/Teacher_model_after_epoch_100_trainLoss_1.6278_valLoss_0.7199.pth'
        }
    
    gli_teacher = teacher_model_paths["GLI"]
    ssa_teacher = teacher_model_paths["SSA"]
    ped_teacher = teacher_model_paths["PED"]
    men_teacher = teacher_model_paths["MEN"]
    met_teacher = teacher_model_paths["MET"]
    
    teacher_models = [gli_teacher, ssa_teacher, ped_teacher, men_teacher, met_teacher]

    return teacher_models

# Testing

In [8]:
from monai.metrics import DiceMetric, HausdorffDistanceMetric

class ComputeMetrics(nn.Module):
    def __init__(self):
        super(ComputeMetrics, self).__init__()
        self.dice_metric = DiceMetric(reduction="mean_batch")
        self.hausdorff_metric = HausdorffDistanceMetric(percentile=95.0, reduction="mean_batch")

    def compute(self, p, y, lbl):
        self.dice_metric.reset()
        self.hausdorff_metric.reset()
        
        print(f"{lbl} - Prediction unique values: {torch.unique(p)}")
        print(f"{lbl} - Ground truth unique values: {torch.unique(y)}")

        if torch.sum(y.float()) == 0 and torch.sum(p.float()) == 0:  # True Negative Case: No foreground pixels in GT
            print(f"{lbl} - No positive samples in ground truth.")
            print(f"Dice scores for {lbl} for this batch: {1.0}")
            print(f"Hausdorff distances for {lbl} for this batch: {0.0}")
            return torch.tensor(1.0), torch.tensor(0.0)
        
        if torch.sum(p.float()) == 0 and torch.sum(y.float()) > 0:  # False Negative Case: GT has 1s, Prediction is all 0s
            print(f"{lbl} - False Negative Case: GT has positive samples, but prediction is empty.")
            print(f"Dice scores for {lbl} for this batch: {0.0}")
            print(f"Hausdorff distances for {lbl} for this batch: {373.1287}")
            return torch.tensor(0.0), torch.tensor(373.1287)
        
        if torch.sum(p.float()) > 0 and torch.sum(y.float()) == 0:  # False Positive Case: Prediction has 1s, GT is all 0s
            print(f"{lbl} - False Positive Case: Prediction has positives, but ground truth is empty.")
            print(f"Dice scores for {lbl} for this batch: {0.0}")
            print(f"Hausdorff distances for {lbl} for this batch: {373.1287}")
            return torch.tensor(0.0), torch.tensor(373.1287)

        # Compute metrics normally
        dice_score = self.dice_metric(p.float(), y.float())
        hausdorff_dist = self.hausdorff_metric(p.float(), y.float())

        print(f"Dice scores for {lbl} for this batch:\n {dice_score.item()}")
        print(f"Hausdorff distances for {lbl} for this batch:\n{hausdorff_dist.item()}")
    
        return dice_score, hausdorff_dist

    def forward(self, p, y):
        # P torch.Size([1, 3, 128, 128, 128])
        # Y torch.Size([1, 1, 128, 128, 128])
        # P_wt torch.Size([1, 1, 128, 128, 128])
        # p = (torch.sigmoid(p) > 0.5)
        y_wt, y_tc, y_et = y > 0, ((y == 1) + (y == 3)) > 0, y == 3
        p_wt, p_tc, p_et = p[:, 0].unsqueeze(1), p[:, 1].unsqueeze(1), p[:, 2].unsqueeze(1)
        
        dice_wt, hd_wt = self.compute(p_wt, y_wt, 'wt')
        dice_tc, hd_tc = self.compute(p_tc, y_tc, 'tc')
        dice_et, hd_et = self.compute(p_et, y_et, 'et')
        
        return [dice_wt, hd_wt], [dice_tc, hd_tc], [dice_et, hd_et]

In [9]:
def test_net(loader, teachers):
    torch.manual_seed(0)
    n_test_batches = len(loader)

    compute_metrics = ComputeMetrics()
    total_metrics = {"WT": {'dice_score': 0, 'hausdorff_distance': 0},
                     "TC": {'dice_score': 0, 'hausdorff_distance': 0},
                     "ET": {'dice_score': 0, 'hausdorff_distance': 0}}

    with tqdm(total=n_test_batches, desc='Testing', unit='batch', leave=False) as pbar:
        with torch.no_grad():
            for step, y in enumerate(loader):
                y['imgs'], y['masks']= y['imgs'].to('cuda'), y['masks'].to('cuda')
                
                with torch.amp.autocast('cuda'):
                    print("--------Now patient:", y['patient_id'])
                    res = run_ensemble_fast(y['imgs'], teachers)
                    wt_metrics, tc_metrics, et_metrics = compute_metrics(res, y['masks'])
                    
                    total_metrics['WT']['dice_score'] += wt_metrics[0].item()
                    total_metrics['WT']['hausdorff_distance'] += wt_metrics[1].item()

                    total_metrics['TC']['dice_score'] += tc_metrics[0].item()
                    total_metrics['TC']['hausdorff_distance'] += tc_metrics[1].item()

                    total_metrics['ET']['dice_score'] += et_metrics[0].item()
                    total_metrics['ET']['hausdorff_distance'] += et_metrics[1].item()
                                    
                pbar.update(1)

        total_metrics['WT']['dice_score'] /= n_test_batches
        total_metrics['WT']['hausdorff_distance'] /= n_test_batches

        total_metrics['TC']['dice_score'] /= n_test_batches
        total_metrics['TC']['hausdorff_distance'] /= n_test_batches

        total_metrics['ET']['dice_score'] /= n_test_batches
        total_metrics['ET']['hausdorff_distance'] /= n_test_batches


        print("************************************************************************")
        print(f"Average Dice Score for WT: {total_metrics['WT']['dice_score']:.4f}")
        print(f"Average Hausdorff Distance for WT: {total_metrics['WT']['hausdorff_distance']:.4f}")

        print("-----------------------------------------------------------------------------")
        print("-----------------------------------------------------------------------------")
                                     
        print(f"Average Dice Score for TC: {total_metrics['TC']['dice_score']:.4f}")
        print(f"Average Hausdorff Distance for TC: {total_metrics['TC']['hausdorff_distance']:.4f}")
                              
        print("-----------------------------------------------------------------------------")
        print("-----------------------------------------------------------------------------")
                                     
        print(f"Average Dice Score for ET: {total_metrics['ET']['dice_score']:.4f}")
        print(f"Average Hausdorff Distance for ET: {total_metrics['ET']['hausdorff_distance']:.4f}")
        print("************************************************************************")
    
    return

## Preparing teachers

In [10]:
teacher_models = prepare_teachers()

## Testing on GLI

In [11]:
args = {
    'workers': 2,
    'train_batch_size': 2,
    'val_batch_size': 2,
    'test_batch_size': 1,
    # 'data_dirs': ["/kaggle/input/bratsglioma/Training/", "/kaggle/input/bratsafrica24/", "/kaggle/input/bratsped/Training/", "/kaggle/input/bratsmen/", "/kaggle/input/bratsmet24/"],
    'data_dirs': ["/kaggle/input/bratsglioma/Training/"],
}

_, _, testLoader = prepare_data_loaders(args)
test_net(testLoader, teacher_models)

Test IDs of /kaggle/input/bratsglioma/Training/ ['BraTS-GLI-00000-000', 'BraTS-GLI-00005-000', 'BraTS-GLI-00008-001', 'BraTS-GLI-00012-000', 'BraTS-GLI-00019-000', 'BraTS-GLI-00022-000', 'BraTS-GLI-00024-001', 'BraTS-GLI-00036-000', 'BraTS-GLI-00048-000', 'BraTS-GLI-00049-000', 'BraTS-GLI-00059-000', 'BraTS-GLI-00061-000', 'BraTS-GLI-00068-000', 'BraTS-GLI-00071-000', 'BraTS-GLI-00074-000', 'BraTS-GLI-00081-000', 'BraTS-GLI-00084-001', 'BraTS-GLI-00095-000', 'BraTS-GLI-00100-000', 'BraTS-GLI-00104-000', 'BraTS-GLI-00111-000', 'BraTS-GLI-00116-000', 'BraTS-GLI-00121-000', 'BraTS-GLI-00123-000', 'BraTS-GLI-00124-000', 'BraTS-GLI-00128-000', 'BraTS-GLI-00139-000', 'BraTS-GLI-00144-000', 'BraTS-GLI-00147-000', 'BraTS-GLI-00160-000', 'BraTS-GLI-00185-000', 'BraTS-GLI-00186-000', 'BraTS-GLI-00192-000', 'BraTS-GLI-00217-000', 'BraTS-GLI-00221-000', 'BraTS-GLI-00227-000', 'BraTS-GLI-00253-000', 'BraTS-GLI-00254-000', 'BraTS-GLI-00269-000', 'BraTS-GLI-00273-000', 'BraTS-GLI-00282-000', 'BraTS-G

Testing:   0%|          | 0/251 [00:00<?, ?batch/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.trans

--------Now patient: ['BraTS-GLI-00000-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080_valLoss_0.5561.pth
Loaded model: /kaggle/input/pednewlabel/Teacher_model_after_epoch_99_trainLoss_1.4512_valLoss_1.0042.pth
Loaded model: /kaggle/input/meningiomateachernewlabels/Teacher_model_after_epoch_85_trainLoss_0.5824_valLoss_0.3318.pth
Loaded model: /kaggle/input/met-teacher-new-labels/Teacher_model_after_epoch_100_trainLoss_1.6278_valLoss_0.7199.pth
Shape of each teacher output: torch.Size([3, 128, 128, 128])
Voting by majority for channel 0..
Voting by majority for channel 1..
Voting by majority for channel 2..
Final ensemble predicion: torch.Size([1, 3, 128, 128, 128])
wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')


Testing:   0%|          | 1/251 [00:24<1:41:46, 24.43s/batch]

Dice scores for wt for this batch:
 0.9230055212974548
Hausdorff distances for wt for this batch:
3.7416574954986572
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.9513516426086426
Hausdorff distances for tc for this batch:
1.4142135381698608
et - Prediction unique values: tensor([0., 1.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for et for this batch:
 0.9032523036003113
Hausdorff distances for et for this batch:
1.4142135381698608
--------Now patient: ['BraTS-GLI-00005-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080_valLoss_0.5561.pth
Loaded model: /kaggle/input/pednewlabel/Teacher_model_after_epoch_99_trainLoss_1.4512_valLoss_1.0042.p

...

Testing:  99%|█████████▉| 249/251 [19:08<00:08,  4.40s/batch]

wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for this batch:
 0.9212457537651062
Hausdorff distances for wt for this batch:
3.316624879837036
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.15487992763519287
Hausdorff distances for tc for this batch:
15.145944595336914
et - Prediction unique values: tensor([0., 1.], device='cuda:0')
et - Ground truth unique values: tensor([False], device='cuda:0')
et - False Positive Case: Prediction has positives, but ground truth is empty.
Dice scores for et for this batch: 0.0
Hausdorff distances for et for this batch: 373.1287
--------Now patient: ['BraTS-GLI-01527-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input

Testing: 100%|█████████▉| 250/251 [19:13<00:04,  4.40s/batch]

wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for this batch:
 0.9525753259658813
Hausdorff distances for wt for this batch:
1.7320507764816284
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.8960665464401245
Hausdorff distances for tc for this batch:
3.0
et - Prediction unique values: tensor([0., 1.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for et for this batch:
 0.05882352963089943
Hausdorff distances for et for this batch:
25.980762481689453
--------Now patient: ['BraTS-GLI-01661-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.10

Dice scores for et for this batch:
 0.931244432926178
Hausdorff distances for et for this batch:
2.2360680103302
************************************************************************
Average Dice Score for WT: 0.8725
Average Hausdorff Distance for WT: 6.4995
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
Average Dice Score for TC: 0.8138
Average Hausdorff Distance for TC: 11.1424
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
Average Dice Score for ET: 0.7749
Average Hausdorff Distance for ET: 16.0439
************************************************************************


## Testing on SSA

In [12]:
args = {
    'workers': 2,
    'train_batch_size': 2,
    'val_batch_size': 2,
    'test_batch_size': 1,
    # 'data_dirs': ["/kaggle/input/bratsglioma/Training/", "/kaggle/input/bratsafrica24/", "/kaggle/input/bratsped/Training/", "/kaggle/input/bratsmen/", "/kaggle/input/bratsmet24/"],
    'data_dirs': ["/kaggle/input/bratsafrica24/"],
}

_, _, testLoader = prepare_data_loaders(args)
test_net(testLoader, teacher_models)

Test IDs of /kaggle/input/bratsafrica24/ ['BraTS-SSA-00009-000', 'BraTS-SSA-00014-000', 'BraTS-SSA-00018-000', 'BraTS-SSA-00037-000', 'BraTS-SSA-00045-000', 'BraTS-SSA-00049-000', 'BraTS-SSA-00055-000', 'BraTS-SSA-00062-000', 'BraTS-SSA-00073-000', 'BraTS-SSA-00091-000', 'BraTS-SSA-00097-000', 'BraTS-SSA-00115-000', 'BraTS-SSA-00122-000', 'BraTS-SSA-00130-000', 'BraTS-SSA-00132-000', 'BraTS-SSA-00148-000', 'BraTS-SSA-00163-000', 'BraTS-SSA-00168-000', 'BraTS-SSA-00170-000', 'BraTS-SSA-00171-000', 'BraTS-SSA-00179-000', 'BraTS-SSA-00181-000', 'BraTS-SSA-00190-000', 'BraTS-SSA-00198-000', 'BraTS-SSA-00201-000', 'BraTS-SSA-00202-000', 'BraTS-SSA-00216-000', 'BraTS-SSA-00218-000', 'BraTS-SSA-00221-000', 'BraTS-SSA-00225-000']
Number of training samples in bratsafrica24 DataSet: 102
Number of validation samples in bratsafrica24 DataSet: 13
Number of testing samples in bratsafrica24 DataSet: 30 
Number of combined training samples 102
Number of combined validation samples 12
Number of combin

Testing:   0%|          | 0/30 [00:00<?, ?batch/s]/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


--------Now patient: ['BraTS-SSA-00009-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080_valLoss_0.5561.pth
Loaded model: /kaggle/input/pednewlabel/Teacher_model_after_epoch_99_trainLoss_1.4512_valLoss_1.0042.pth
Loaded model: /kaggle/input/meningiomateachernewlabels/Teacher_model_after_epoch_85_trainLoss_0.5824_valLoss_0.3318.pth
Loaded model: /kaggle/input/met-teacher-new-labels/Teacher_model_after_epoch_100_trainLoss_1.6278_valLoss_0.7199.pth
Shape of each teacher output: torch.Size([3, 128, 128, 128])
Voting by majority for channel 0..
Voting by majority for channel 1..
Voting by majority for channel 2..
Final ensemble predicion: torch.Size([1, 3, 128, 128, 128])


Testing:   3%|▎         | 1/30 [00:07<03:49,  7.92s/batch]

wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for this batch:
 0.9260385036468506
Hausdorff distances for wt for this batch:
1.7320507764816284
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.9192603826522827
Hausdorff distances for tc for this batch:
2.0
et - Prediction unique values: tensor([0., 1.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for et for this batch:
 0.8846100568771362
Hausdorff distances for et for this batch:
4.582575798034668
--------Now patient: ['BraTS-SSA-00014-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080

...

Testing:  97%|█████████▋| 29/30 [02:15<00:04,  4.43s/batch]

wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for this batch:
 0.9367057681083679
Hausdorff distances for wt for this batch:
2.2360680103302
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.5265644788742065
Hausdorff distances for tc for this batch:
26.214479446411133
et - Prediction unique values: tensor([0., 1.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for et for this batch:
 0.8623256087303162
Hausdorff distances for et for this batch:
20.423025131225586
--------Now patient: ['BraTS-SSA-00225-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_tra

Testing: 100%|██████████| 30/30 [02:20<00:00,  4.39s/batch]

wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for this batch:
 0.9242926239967346
Hausdorff distances for wt for this batch:
3.0
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.7649616599082947
Hausdorff distances for tc for this batch:
13.34166431427002
et - Prediction unique values: tensor([0., 1.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for et for this batch:
 0.8890427350997925
Hausdorff distances for et for this batch:
9.523621559143066


************************************************************************
Average Dice Score for WT: 0.8830
Average Hausdorff Distance for WT: 4.7243
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
Average Dice Score for TC: 0.7736
Average Hausdorff Distance for TC: 9.2580
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
Average Dice Score for ET: 0.7519
Average Hausdorff Distance for ET: 18.4846
************************************************************************


## Testing on PED

In [13]:
args = {
    'workers': 2,
    'train_batch_size': 2,
    'val_batch_size': 2,
    'test_batch_size': 1,
    # 'data_dirs': ["/kaggle/input/bratsglioma/Training/", "/kaggle/input/bratsafrica24/", "/kaggle/input/bratsped/Training/", "/kaggle/input/bratsmen/", "/kaggle/input/bratsmet24/"],
    'data_dirs': ["/kaggle/input/bratsped/Training/"],
}

_, _, testLoader = prepare_data_loaders(args)
test_net(testLoader, teacher_models)

Test IDs of /kaggle/input/bratsped/Training/ ['BraTS-PED-00006-000', 'BraTS-PED-00010-000', 'BraTS-PED-00026-000', 'BraTS-PED-00032-000', 'BraTS-PED-00042-000', 'BraTS-PED-00061-000', 'BraTS-PED-00062-000', 'BraTS-PED-00084-000', 'BraTS-PED-00086-000', 'BraTS-PED-00087-000', 'BraTS-PED-00106-000', 'BraTS-PED-00107-000', 'BraTS-PED-00114-000', 'BraTS-PED-00116-000', 'BraTS-PED-00120-000', 'BraTS-PED-00126-000', 'BraTS-PED-00130-000', 'BraTS-PED-00135-000', 'BraTS-PED-00136-000', 'BraTS-PED-00140-000', 'BraTS-PED-00142-000']
Number of training samples in bratsped DataSet: 70
Number of validation samples in bratsped DataSet: 8
Number of testing samples in bratsped DataSet: 21 
Number of combined training samples 70
Number of combined validation samples 8
Number of combined testing samples 21


Testing:   0%|          | 0/21 [00:00<?, ?batch/s]/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


--------Now patient: ['BraTS-PED-00006-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080_valLoss_0.5561.pth
Loaded model: /kaggle/input/pednewlabel/Teacher_model_after_epoch_99_trainLoss_1.4512_valLoss_1.0042.pth
Loaded model: /kaggle/input/meningiomateachernewlabels/Teacher_model_after_epoch_85_trainLoss_0.5824_valLoss_0.3318.pth
Loaded model: /kaggle/input/met-teacher-new-labels/Teacher_model_after_epoch_100_trainLoss_1.6278_valLoss_0.7199.pth
Shape of each teacher output: torch.Size([3, 128, 128, 128])
Voting by majority for channel 0..
Voting by majority for channel 1..
Voting by majority for channel 2..
Final ensemble predicion: torch.Size([1, 3, 128, 128, 128])


Testing:   5%|▍         | 1/21 [00:08<02:40,  8.03s/batch]

wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for this batch:
 0.8912134170532227
Hausdorff distances for wt for this batch:
2.4494898319244385
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.9206750392913818
Hausdorff distances for tc for this batch:
2.2360680103302
et - Prediction unique values: tensor([0., 1.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for et for this batch:
 0.8241708278656006
Hausdorff distances for et for this batch:
2.2360680103302
--------Now patient: ['BraTS-PED-00010-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainL

...

Testing:  95%|█████████▌| 20/21 [01:34<00:04,  4.40s/batch]

wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for this batch:
 0.9163686633110046
Hausdorff distances for wt for this batch:
2.4494898319244385
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.39479437470436096
Hausdorff distances for tc for this batch:
9.0
et - Prediction unique values: tensor([0., 1.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for et for this batch:
 0.27591803669929504
Hausdorff distances for et for this batch:
11.48912525177002
--------Now patient: ['BraTS-PED-00142-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.10

Testing: 100%|██████████| 21/21 [01:39<00:00,  4.39s/batch]

wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for this batch:
 0.830410897731781
Hausdorff distances for wt for this batch:
8.66025447845459
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.0004922168445773423
Hausdorff distances for tc for this batch:
34.41656494140625
et - Prediction unique values: tensor([0.], device='cuda:0')
et - Ground truth unique values: tensor([False], device='cuda:0')
et - No positive samples in ground truth.
Dice scores for et for this batch: 1.0
Hausdorff distances for et for this batch: 0.0


************************************************************************
Average Dice Score for WT: 0.8856
Average Hausdorff Distance for WT: 5.5459
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
Average Dice Score for TC: 0.4639
Average Hausdorff Distance for TC: 12.5607
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
Average Dice Score for ET: 0.4564
Average Hausdorff Distance for ET: 79.4959
************************************************************************


## Testing on MEN

In [14]:
args = {
    'workers': 2,
    'train_batch_size': 2,
    'val_batch_size': 2,
    'test_batch_size': 1,
    # 'data_dirs': ["/kaggle/input/bratsglioma/Training/", "/kaggle/input/bratsafrica24/", "/kaggle/input/bratsped/Training/", "/kaggle/input/bratsmen/", "/kaggle/input/bratsmet24/"],
    'data_dirs': ["/kaggle/input/bratsmen/"],
}

_, _, testLoader = prepare_data_loaders(args)
test_net(testLoader, teacher_models)

Test IDs of /kaggle/input/bratsmen/ ['BraTS-MEN-00008-000', 'BraTS-MEN-00012-000', 'BraTS-MEN-00017-000', 'BraTS-MEN-00022-000', 'BraTS-MEN-00026-000', 'BraTS-MEN-00033-000', 'BraTS-MEN-00041-000', 'BraTS-MEN-00042-000', 'BraTS-MEN-00053-000', 'BraTS-MEN-00057-000', 'BraTS-MEN-00059-000', 'BraTS-MEN-00065-000', 'BraTS-MEN-00074-005', 'BraTS-MEN-00074-007', 'BraTS-MEN-00076-000', 'BraTS-MEN-00083-000', 'BraTS-MEN-00084-000', 'BraTS-MEN-00096-000', 'BraTS-MEN-00101-000', 'BraTS-MEN-00108-004', 'BraTS-MEN-00114-000', 'BraTS-MEN-00122-000', 'BraTS-MEN-00144-000', 'BraTS-MEN-00146-000', 'BraTS-MEN-00161-000', 'BraTS-MEN-00168-000', 'BraTS-MEN-00174-000', 'BraTS-MEN-00181-000', 'BraTS-MEN-00198-000', 'BraTS-MEN-00204-000', 'BraTS-MEN-00207-000', 'BraTS-MEN-00216-000', 'BraTS-MEN-00225-000', 'BraTS-MEN-00227-000', 'BraTS-MEN-00229-000', 'BraTS-MEN-00238-000', 'BraTS-MEN-00244-000', 'BraTS-MEN-00246-000', 'BraTS-MEN-00256-000', 'BraTS-MEN-00258-000', 'BraTS-MEN-00276-000', 'BraTS-MEN-00278-000

Testing:   0%|          | 0/200 [00:00<?, ?batch/s]/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


--------Now patient: ['BraTS-MEN-00008-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080_valLoss_0.5561.pth
Loaded model: /kaggle/input/pednewlabel/Teacher_model_after_epoch_99_trainLoss_1.4512_valLoss_1.0042.pth
Loaded model: /kaggle/input/meningiomateachernewlabels/Teacher_model_after_epoch_85_trainLoss_0.5824_valLoss_0.3318.pth
Loaded model: /kaggle/input/met-teacher-new-labels/Teacher_model_after_epoch_100_trainLoss_1.6278_valLoss_0.7199.pth
Shape of each teacher output: torch.Size([3, 128, 128, 128])
Voting by majority for channel 0..
Voting by majority for channel 1..
Voting by majority for channel 2..
Final ensemble predicion: torch.Size([1, 3, 128, 128, 128])


Testing:   0%|          | 1/200 [00:07<24:50,  7.49s/batch]

wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for this batch:
 0.8822078704833984
Hausdorff distances for wt for this batch:
2.0
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.9393048882484436
Hausdorff distances for tc for this batch:
1.0
et - Prediction unique values: tensor([0., 1.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for et for this batch:
 0.942740261554718
Hausdorff distances for et for this batch:
1.0
--------Now patient: ['BraTS-MEN-00012-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080_valLoss_0.5561.pth
Loaded mod

...

Testing:  99%|█████████▉| 198/200 [15:05<00:08,  4.44s/batch]

wt - Prediction unique values: tensor([0.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
wt - False Negative Case: GT has positive samples, but prediction is empty.
Dice scores for wt for this batch: 0.0
Hausdorff distances for wt for this batch: 373.1287
tc - Prediction unique values: tensor([0.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
tc - False Negative Case: GT has positive samples, but prediction is empty.
Dice scores for tc for this batch: 0.0
Hausdorff distances for tc for this batch: 373.1287
et - Prediction unique values: tensor([0.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
et - False Negative Case: GT has positive samples, but prediction is empty.
Dice scores for et for this batch: 0.0
Hausdorff distances for et for this batch: 373.1287
--------Now patient: ['BraTS-MEN-01430-000']
Loaded model: /kaggle/input/gliomateachernewlabels/T

Testing: 100%|█████████▉| 199/200 [15:09<00:04,  4.35s/batch]

wt - Prediction unique values: tensor([0.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
wt - False Negative Case: GT has positive samples, but prediction is empty.
Dice scores for wt for this batch: 0.0
Hausdorff distances for wt for this batch: 373.1287
tc - Prediction unique values: tensor([0.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
tc - False Negative Case: GT has positive samples, but prediction is empty.
Dice scores for tc for this batch: 0.0
Hausdorff distances for tc for this batch: 373.1287
et - Prediction unique values: tensor([0.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
et - False Negative Case: GT has positive samples, but prediction is empty.
Dice scores for et for this batch: 0.0
Hausdorff distances for et for this batch: 373.1287
--------Now patient: ['BraTS-MEN-01431-000']
Loaded model: /kaggle/input/gliomateachernewlabels/T

wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for this batch:
 0.8481444716453552
Hausdorff distances for wt for this batch:
2.4494898319244385
tc - Prediction unique values: tensor([0., 1.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for tc for this batch:
 0.7949094176292419
Hausdorff distances for tc for this batch:
8.5440034866333
et - Prediction unique values: tensor([0., 1.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for et for this batch:
 0.9157593250274658
Hausdorff distances for et for this batch:
1.0
************************************************************************
Average Dice Score for WT: 0.6679
Average Hausdorff Distance for WT: 55.4825
-----------------------------------------------------------------------------
-------------------------

## Testing on MET

In [15]:
args = {
    'workers': 2,
    'train_batch_size': 2,
    'val_batch_size': 2,
    'test_batch_size': 1,
    # 'data_dirs': ["/kaggle/input/bratsglioma/Training/", "/kaggle/input/bratsafrica24/", "/kaggle/input/bratsped/Training/", "/kaggle/input/bratsmen/", "/kaggle/input/bratsmet24/"],
    'data_dirs': ["/kaggle/input/bratsmet24/"],
}

_, _, testLoader = prepare_data_loaders(args)
test_net(testLoader, teacher_models)

Test IDs of /kaggle/input/bratsmet24/ ['BraTS-MET-00001-000', 'BraTS-MET-00002-000', 'BraTS-MET-00003-000', 'BraTS-MET-00007-000', 'BraTS-MET-00012-000', 'BraTS-MET-00015-000', 'BraTS-MET-00027-000', 'BraTS-MET-00030-000', 'BraTS-MET-00034-000', 'BraTS-MET-00037-000', 'BraTS-MET-00111-000', 'BraTS-MET-00116-000', 'BraTS-MET-00123-000', 'BraTS-MET-00126-000', 'BraTS-MET-00130-000', 'BraTS-MET-00133-000', 'BraTS-MET-00140-000', 'BraTS-MET-00141-000', 'BraTS-MET-00144-000', 'BraTS-MET-00145-000', 'BraTS-MET-00151-000', 'BraTS-MET-00154-000', 'BraTS-MET-00155-000', 'BraTS-MET-00161-000', 'BraTS-MET-00162-000', 'BraTS-MET-00165-000', 'BraTS-MET-00169-000', 'BraTS-MET-00172-000', 'BraTS-MET-00189-000', 'BraTS-MET-00190-000', 'BraTS-MET-00193-000', 'BraTS-MET-00199-000', 'BraTS-MET-00207-000', 'BraTS-MET-00212-000', 'BraTS-MET-00216-000', 'BraTS-MET-00219-000', 'BraTS-MET-00221-000', 'BraTS-MET-00223-000', 'BraTS-MET-00225-000', 'BraTS-MET-00226-000', 'BraTS-MET-00237-000', 'BraTS-MET-00241-0

Testing:   0%|          | 0/67 [00:00<?, ?batch/s]/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)
/opt/conda/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


--------Now patient: ['BraTS-MET-00001-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080_valLoss_0.5561.pth
Loaded model: /kaggle/input/pednewlabel/Teacher_model_after_epoch_99_trainLoss_1.4512_valLoss_1.0042.pth
Loaded model: /kaggle/input/meningiomateachernewlabels/Teacher_model_after_epoch_85_trainLoss_0.5824_valLoss_0.3318.pth
Loaded model: /kaggle/input/met-teacher-new-labels/Teacher_model_after_epoch_100_trainLoss_1.6278_valLoss_0.7199.pth
Shape of each teacher output: torch.Size([3, 128, 128, 128])
Voting by majority for channel 0..
Voting by majority for channel 1..
Voting by majority for channel 2..
Final ensemble predicion: torch.Size([1, 3, 128, 128, 128])
wt - Prediction unique values: tensor([0., 1.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
Dice scores for wt for 

Testing:   1%|▏         | 1/67 [00:08<09:42,  8.83s/batch]

Dice scores for et for this batch:
 0.4643864929676056
Hausdorff distances for et for this batch:
61.514225006103516
--------Now patient: ['BraTS-MET-00002-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080_valLoss_0.5561.pth
Loaded model: /kaggle/input/pednewlabel/Teacher_model_after_epoch_99_trainLoss_1.4512_valLoss_1.0042.pth
Loaded model: /kaggle/input/meningiomateachernewlabels/Teacher_model_after_epoch_85_trainLoss_0.5824_valLoss_0.3318.pth
Loaded model: /kaggle/input/met-teacher-new-labels/Teacher_model_after_epoch_100_trainLoss_1.6278_valLoss_0.7199.pth
Shape of each teacher output: torch.Size([3, 128, 128, 128])
Voting by majority for channel 0..
Voting by majority for channel 1..
Voting by majority for channel 2..
Final ensemble predicion: torch.Size([1, 3, 128, 128, 128])
wt - Prediction unique values: tensor([0., 1

...

Testing:  97%|█████████▋| 65/67 [05:06<00:09,  4.62s/batch]

Dice scores for et for this batch:
 0.5601038932800293
Hausdorff distances for et for this batch:
38.55902862548828
--------Now patient: ['BraTS-MET-00813-000']
Loaded model: /kaggle/input/gliomateachernewlabels/Teacher_model_after_epoch_100_trainLoss_0.5972_valLoss_0.3019.pth
Loaded model: /kaggle/input/africanewlabels/Teacher_model_after_epoch_67_trainLoss_1.1080_valLoss_0.5561.pth
Loaded model: /kaggle/input/pednewlabel/Teacher_model_after_epoch_99_trainLoss_1.4512_valLoss_1.0042.pth
Loaded model: /kaggle/input/meningiomateachernewlabels/Teacher_model_after_epoch_85_trainLoss_0.5824_valLoss_0.3318.pth
Loaded model: /kaggle/input/met-teacher-new-labels/Teacher_model_after_epoch_100_trainLoss_1.6278_valLoss_0.7199.pth
Shape of each teacher output: torch.Size([3, 128, 128, 128])
Voting by majority for channel 0..
Voting by majority for channel 1..
Voting by majority for channel 2..
Final ensemble predicion: torch.Size([1, 3, 128, 128, 128])


Testing:  99%|█████████▊| 66/67 [05:11<00:04,  4.60s/batch]

wt - Prediction unique values: tensor([0.], device='cuda:0')
wt - Ground truth unique values: tensor([False,  True], device='cuda:0')
wt - False Negative Case: GT has positive samples, but prediction is empty.
Dice scores for wt for this batch: 0.0
Hausdorff distances for wt for this batch: 373.1287
tc - Prediction unique values: tensor([0.], device='cuda:0')
tc - Ground truth unique values: tensor([False,  True], device='cuda:0')
tc - False Negative Case: GT has positive samples, but prediction is empty.
Dice scores for tc for this batch: 0.0
Hausdorff distances for tc for this batch: 373.1287
et - Prediction unique values: tensor([0.], device='cuda:0')
et - Ground truth unique values: tensor([False,  True], device='cuda:0')
et - False Negative Case: GT has positive samples, but prediction is empty.
Dice scores for et for this batch: 0.0
Hausdorff distances for et for this batch: 373.1287
--------Now patient: ['BraTS-MET-00816-000']
Loaded model: /kaggle/input/gliomateachernewlabels/T

Dice scores for et for this batch:
 0.7562847137451172
Hausdorff distances for et for this batch:
3.5984373092651367
************************************************************************
Average Dice Score for WT: 0.5604
Average Hausdorff Distance for WT: 88.8783
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
Average Dice Score for TC: 0.4750
Average Hausdorff Distance for TC: 111.9880
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
Average Dice Score for ET: 0.4270
Average Hausdorff Distance for ET: 111.7110
************************************************************************


# Press here